In [1]:
import sys
from qwikidata.sparql import return_sparql_query_results
from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api
from SPARQLWrapper import SPARQLWrapper, JSON
import json
from tqdm import tqdm
import re
import requests as r
import pandas as pd
from collections import defaultdict
from libindic import inexactsearch
import pandas as pd
import pickle as pkl
import wptools
sys.path.append('..')
from method2.People_translator import People_translator
translator = People_translator()
from method1.utils import *
from baseline.baseline import baseline_infobox
from method1.get_infobox import method1_infobox

In [2]:
with open('../data-collection/persons/primary_dataset_new_withlinks.json') as f:
    data = json.loads(f.read())
for ind , val in enumerate(data):
    val['index'] = ind

In [4]:
%timeit
for entry in data:
    try :
        print(entry['wd_id'] , entry['en_wikipedia_title'])
        baseline = baseline_infobox(entry['wd_id'] ,category='people')
        method2 = translator.get_infobox(entry['en_wikipedia_title'] , test=True)
        method1 = method1_infobox(entry['wd_id'],'people')
        output = {
            "method_1" : method1,
            "method_2" : method2,
            "baseline" : baseline
        }
        with open('people_output.jsonl' , 'a+') as f:
            f.write(json.dumps({entry['index'] : output} , ensure_ascii = False))
            f.write("\n")
    except Exception as e:
        print(e)
        

Q555240 Shane Warne


en.wikipedia.org (parse) Shane Warne
en.wikipedia.org (imageinfo) File:Shane Warne February 2015.jpg
Shane Warne (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Shane War...
  infobox: <dict(97)> name, image, caption, country, fullname, nic...
  iwlinks: <list(2)> https://commons.wikimedia.org/wiki/Category:S...
  pageid: 22590160
  parsetree: <str(118605)> <root><template><title>pp-pc1</title></...
  requests: <list(2)> parse, imageinfo
  title: Shane Warne
  wikibase: Q555240
  wikidata_url: https://www.wikidata.org/wiki/Q555240
  wikitext: <str(94508)> {{pp-pc1}}{{Short description|Australian ...
}
